In [30]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier # Import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix




In [31]:
data = pd.read_csv('/content/Bank_customer_data.csv')

data.head(3)

,ID,Age,Experience,Income,Zipcode,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Personal Loan
0,1,39,14,127000,49560,1,567.28,2,36000,1,0,1,0,1
1,2,36,0,72000,63865,1,3927.17,1,341000,1,0,0,0,0
2,3,37,5,147000,56686,2,369.79,1,326000,0,0,0,1,0


In [32]:
# Drop unnecessary columns
data.drop(columns=['ID', 'Zipcode'], inplace=True)

X = data.drop(columns=['Personal Loan'])  # All features except target
y = data['Personal Loan']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print("X_train Shape:", X_train.shape)
print("X_test Shape:", X_test.shape)

X_train Shape: (320, 11)
X_test Shape: (100, 11)


In [34]:

# Function to create the model (Fixed Input Layer)
def create_model(learning_rate=0.01, num_neurons=32, dropout_rate=0.2):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # Explicit Input Layer
        Dense(num_neurons, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_neurons, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model in KerasClassifier and explicitly pass default parameters
model = KerasClassifier(model=create_model, verbose=0)

# Define the hyperparameter grid (use 'model__' prefix for parameters)
param_grid = {
    'model__learning_rate': [0.001, 0.01, 0.1],
    'model__num_neurons': [16, 32, 64],
    'model__dropout_rate': [0.2, 0.4, 0.5],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100]
}

# Perform Randomized Search
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=3, verbose=1, n_jobs=-1)
random_search.fit(X_train, y_train)

# Best parameters
print("Best Hyperparameters:", random_search.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Hyperparameters: {'model__num_neurons': 64, 'model__learning_rate': 0.1, 'model__dropout_rate': 0.5, 'epochs': 100, 'batch_size': 64}
